In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import cv2
import torch
import subprocess
import os

# Add the directory containing RRDBNet_arch.py to sys.path if not added already
import sys
sys.path.append(r'C:\Users\ilang\OneDrive\Desktop\pix\ESRGAN')

# Import your architecture file for ESRGAN
import RRDBNet_arch as arch

# Define image dimensions
img_height, img_width = 128, 128

# Load the pre-trained detection model
detection_model_path = r'C:\Users\ilang\OneDrive\Desktop\pix\pixelated_image_detector.h5'
detection_model = load_model(detection_model_path)

# Compile the detection model
detection_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Load the pre-trained ESRGAN model for super-resolution using PyTorch
esrgan_model_path = r'C:\Users\ilang\OneDrive\Desktop\pix\ESRGAN\models\RRDB_ESRGAN_x4.pth'
esrgan_model = arch.RRDBNet(3, 3, 64, 23, gc=32)
esrgan_model.load_state_dict(torch.load(esrgan_model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
esrgan_model.eval()  # Set the model to evaluation mode
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
esrgan_model = esrgan_model.to(device)

def preprocess_image(image_path):
    """ Preprocess the image for pixelation detection. """
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (img_width, img_height))
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)
    return image

class PixelatedImageApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Pixelated Image Detection and Correction")
        self.root.geometry("1200x800")
        self.root.configure(bg='pink')
        
        self.load_image_button = tk.Button(self.root, text="Load Image", command=self.load_image, bg='lightblue', width=20, height=2)
        self.load_image_button.pack(pady=20)
        
        self.pixelation_status_label = tk.Label(self.root, text="Pixelation Status: ",bg='pink')
        self.pixelation_status_label.pack(pady=10)
        
        self.detect_button = tk.Button(self.root, text="Detect Pixelation", command=self.detect_pixelation, bg='lightblue', width=20, height=2)
        self.detect_button.pack(pady=10)
        
        self.correct_button = tk.Button(self.root, text="Correct Image", command=self.correct_image, bg='lightblue', width=20, height=2)
        self.correct_button.pack(pady=10)
        
        self.original_image_label = tk.Label(self.root)
        self.original_image_label.pack(pady=20, side=tk.LEFT)
        
        self.corrected_image_label = tk.Label(self.root)
        self.corrected_image_label.pack(pady=20, side=tk.RIGHT)
        
        self.result_label = tk.Label(self.root, text="")
        self.result_label.pack(pady=10)
        
        self.image_path = None
        self.original_image = None
        self.corrected_image = None
    
    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
        if file_path:
            self.image_path = file_path
            self.original_image = Image.open(file_path)
            self.display_image(self.original_image, self.original_image_label)
            self.result_label.config(text="")
            self.pixelation_status_label.config(text="Pixelation Status: ")
    
    def display_image(self, image, label):
        image = image.resize((400, 300))  # Resize for display if needed
        img = ImageTk.PhotoImage(image)
        label.config(image=img)
        label.image = img  # Keep a reference to prevent garbage collection
        
    def detect_pixelation(self):
        if self.image_path:
            try:
                processed_image = preprocess_image(self.image_path)
                prediction = detection_model.predict(processed_image)
                
                if prediction > 0.5:
                    self.pixelation_status_label.config(text="Pixelation Status: Pixelated")
                    self.result_label.config(text="Image is Pixelated!")
                else:
                    self.pixelation_status_label.config(text="Pixelation Status: Not Pixelated")
                    self.result_label.config(text="Image is not Pixelated.")
            except Exception as e:
                messagebox.showerror("Error", f"Error detecting pixelation: {str(e)}")
        else:
            messagebox.showerror("Error", "Please load an image first.")
    
    def correct_image(self):
        if self.image_path:
            try:
                # Correct with ESRGAN
                corrected_image_esrgan = self.correct_with_esrgan()
                
                # Display ESRGAN corrected image
                esrgan_image = Image.fromarray(cv2.cvtColor(corrected_image_esrgan, cv2.COLOR_BGR2RGB))
                self.display_image(esrgan_image, self.corrected_image_label)
                self.result_label.config(text="Image is Corrected with ESRGAN!")
                
                # Save the corrected ESRGAN image (optional)
                corrected_image_path_esrgan = r'C:\Users\ilang\OneDrive\Desktop\pix\corrected_image_esrgan.jpg'
                cv2.imwrite(corrected_image_path_esrgan, corrected_image_esrgan)
                print(f'ESRGAN Corrected image saved at {corrected_image_path_esrgan}')
                
                # Correct with Real-ESRGAN
                corrected_image_real_esrgan = self.correct_with_real_esrgan()
                
                # Display Real-ESRGAN corrected image
                real_esrgan_image = Image.fromarray(cv2.cvtColor(corrected_image_real_esrgan, cv2.COLOR_BGR2RGB))
                self.display_image(real_esrgan_image, self.corrected_image_label)
                self.result_label.config(text="Image is Corrected with Real-ESRGAN!")
                
                # Save the corrected Real-ESRGAN image (optional)
                corrected_image_path_real_esrgan = r'C:\Users\ilang\OneDrive\Desktop\pix\corrected_image_real_esrgan.jpg'
                cv2.imwrite(corrected_image_path_real_esrgan, corrected_image_real_esrgan)
                print(f'Real-ESRGAN Corrected image saved at {corrected_image_path_real_esrgan}')
                
            except Exception as e:
                messagebox.showerror("Error", f"Error correcting image: {str(e)}")
        else:
            messagebox.showerror("Error", "Please load an image first.")
    
    def correct_with_esrgan(self):
        img = cv2.imread(self.image_path, cv2.IMREAD_COLOR)
        img = img.astype(np.float32) / 255.0
        img = torch.from_numpy(np.transpose(img, (2, 0, 1))).unsqueeze(0).to(device)
        
        # Downscale the image if it's too large
        if img.shape[2] > 512 or img.shape[3] > 512:
            print("Downscaling the image to fit into memory.")
            img = torch.nn.functional.interpolate(img, size=(512, 512), mode='bicubic')
        
        with torch.no_grad():
            output = esrgan_model(img).squeeze().cpu().clamp_(0, 1).numpy()
        
        corrected_image = np.transpose(output, (1, 2, 0))
        corrected_image = (corrected_image * 255.0).round().astype(np.uint8)
        print("Corrected using ESRGAN")
        return corrected_image
    
    def correct_with_real_esrgan(self):
        real_esrgan_exe = r"C:\Users\ilang\OneDrive\Desktop\pix\Real_ESRGAN\realesrgan-ncnn-vulkan.exe"
        command = [real_esrgan_exe, '-i', self.image_path, '-o', 'output_image.png']

        try:
            subprocess.run(command, shell=True, check=True)
            
            # Load and return the enhanced image
            enhanced_image = cv2.imread('output_image.png', cv2.IMREAD_COLOR)
            if enhanced_image is None:
                raise ValueError("Error: Real-ESRGAN output image not found or unable to read.")
            
            os.remove('output_image.png')  # Clean up temporary output file
            
            return enhanced_image
        
        except subprocess.CalledProcessError as e:
            raise RuntimeError(f"Error running Real-ESRGAN: {e}")

# Function to start the Tkinter GUI
def start_gui():
    root = tk.Tk()
    app = PixelatedImageApp(root)
    root.mainloop()

# Start the Tkinter GUI
start_gui()


In [ ]:
import os
from sklearn.metrics import accuracy_score, f1_score
import cv2

def preprocess_image(image_path):
    """ Preprocess the image for pixelation detection. """
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Error: Unable to read image at {image_path}")
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (img_width, img_height))
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

def evaluate_detection_model(test_images_dir):
    """ Evaluate the detection model on a test dataset. """
    y_true = []
    y_pred = []

    # Ensure the directory exists
    if not os.path.exists(test_images_dir):
        print(f"Test images directory does not exist: {test_images_dir}")
        return

    # Ensure the directory is not empty
    if not os.listdir(test_images_dir):
        print(f"Test images directory is empty: {test_images_dir}")
        return

    for image_name in os.listdir(test_images_dir):
        image_path = os.path.join(test_images_dir, image_name)
        print(f"Processing image: {image_path}")  # Debugging statement

        label = 1 if 'pixelated' in image_name else 0

        try:
            processed_image = preprocess_image(image_path)
        except ValueError as e:
            print(e)
            continue

        prediction = detection_model.predict(processed_image)

        y_true.append(label)
        y_pred.append(1 if prediction > 0.5 else 0)

        # Debugging statement to show the processing status
        print(f"Processed {image_name}, True label: {label}, Prediction: {prediction}")

    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f"Detection Model Accuracy: {accuracy:.4f}")
   

# Example usage
test_images_dir = r'C:\Users\ilang\OneDrive\Desktop\pix\test_images'
evaluate_detection_model(test_images_dir)



In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# Assuming y_true and y_pred are your true and predicted labels, respectively
# Replace these with your actual true and predicted labels
y_true = np.array([1, 0, 0, 1, 1, 1, 0, 1, 0, 0])
y_pred = np.array([1, 0, 1, 1, 0, 1, 0, 1, 0, 1])

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Print confusion matrix (optional)
print("Confusion Matrix:")
print(cm)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

classes = np.unique(y_true)
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

thresh = cm.max() / 2.
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()
